In [2]:
#%pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 --force-reinstall

In [3]:
"""import torch

def format_pytorch_version(version_str):
    # Example input: "2.0.1+cu118" -> returns "2.0.1"
    return version_str.split('+')[0]

def format_cuda_version(cuda_str):
    # If CUDA is None (CPU-only PyTorch), return "cpu"
    if cuda_str is None:
        return "cpu"
    # Example: "11.8" -> "cu118"
    return "cu" + cuda_str.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

print(f"Installing packages for torch-{TORCH}+{CUDA}...")

%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

%pip install torch_geometric"""

'import torch\n\ndef format_pytorch_version(version_str):\n    # Example input: "2.0.1+cu118" -> returns "2.0.1"\n    return version_str.split(\'+\')[0]\n\ndef format_cuda_version(cuda_str):\n    # If CUDA is None (CPU-only PyTorch), return "cpu"\n    if cuda_str is None:\n        return "cpu"\n    # Example: "11.8" -> "cu118"\n    return "cu" + cuda_str.replace(\'.\', \'\')\n\nTORCH_version = torch.__version__\nTORCH = format_pytorch_version(TORCH_version)\nCUDA_version = torch.version.cuda\nCUDA = format_cuda_version(CUDA_version)\n\nprint(f"Installing packages for torch-{TORCH}+{CUDA}...")\n\n%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html\n\n%pip install torch_geometric'

In [4]:
#%pip install scikit-learn imbalanced-learn xgboost

In [5]:
#%pip install optuna

### Hyperparameter Optimization Citeseer
#### SCAR 

In [11]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 9),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.12, 0.3),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs": 10,#trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [1024,2048]),
        "lr":0.005,# trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "citeseer_scar_val.csv",
        "min":0.82,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["sage"])#,"cluster"])#,"shine"]),         
        
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-14 18:30:10,777] A new study created in memory with name: no-name-6218f7c5-55fe-42de-9f36-3ffdd0c63d7e


Running experiment with seed=654:
 - K=5, layers=1, hidden=64, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.15820317681000245, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 2.0119
Epoch 10 / 100, Loss: 1.8634
Epoch 20 / 100, Loss: 1.8153
Epoch 30 / 100, Loss: 1.7032
Epoch 40 / 100, Loss: 1.6629
Epoch 50 / 100, Loss: 1.6020
Epoch 60 / 100, Loss: 1.4991
Epoch 70 / 100, Loss: 1.3861
Epoch 80 / 100, Loss: 1.3178
Epoch 90 / 100, Loss: 1.2314


[I 2025-04-14 18:30:44,781] Trial 0 finished with value: 0.7777777777777778 and parameters: {'K': 5, 'layers': 1, 'hidden_channels': 64, 'out_channels': 64, 'ratio': 0.15820317681000245, 'model_type': 'GATConv', 'batch_size': 2048, 'sampling': 'sage'}. Best is trial 0 with value: 0.7777777777777778.


 - Test Metrics: Accuracy=0.9246, F1=0.8158, Recall=0.7932, Precision=0.8399
 - Validation Metrics: Accuracy=0.6364, F1=0.7778, Recall=0.6364, Precision=1.0000
F1 = 0.78 < 0.82, skipping ...
Done. Results written to citeseer_experimentations\citeseer_scar_val_1404183010.csv.
Average F1 over valid seeds: 0.7778 ± 0.0000
Running experiment with seed=654:
 - K=5, layers=1, hidden=64, out=64
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.2717284738811454, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 4.3151
Epoch 10 / 100, Loss: 3.7936
Epoch 20 / 100, Loss: 3.5168
Epoch 30 / 100, Loss: 3.0444
Epoch 40 / 100, Loss: 2.8020
Epoch 50 / 100, Loss: 2.4929
Epoch 60 / 100, Loss: 2.1020
Epoch 70 / 100, Loss: 1.8696
Epoch 80 / 100, Loss: 1.5745
Epoch 90 / 100, Loss: 1.4183
 - Test Metrics: Accuracy=0.9170, F1=0.8110, Recall=0.8445, Precision=0.7800
 - Valid

[I 2025-04-14 18:32:28,897] Trial 1 finished with value: 0.8416540376704914 and parameters: {'K': 5, 'layers': 1, 'hidden_channels': 64, 'out_channels': 64, 'ratio': 0.2717284738811454, 'model_type': 'GCNConv', 'batch_size': 1024, 'sampling': 'sage'}. Best is trial 1 with value: 0.8416540376704914.


...

 - Test Metrics: Accuracy=0.9276, F1=0.8386, Recall=0.8930, Precision=0.7904
 - Validation Metrics: Accuracy=0.8068, F1=0.8931, Recall=0.8068, Precision=1.0000
Done. Results written to citeseer_experimentations\citeseer_scar_val_1404203216.csv.
Average F1 over valid seeds: 0.8773 ± 0.0165
Running experiment with seed=654:
 - K=6, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.14527171269867037, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 4.5929
Epoch 10 / 100, Loss: 4.2546
Epoch 20 / 100, Loss: 3.7899
Epoch 30 / 100, Loss: 3.4862
Epoch 40 / 100, Loss: 3.0989
Epoch 50 / 100, Loss: 2.7091
Epoch 60 / 100, Loss: 2.4040
Epoch 70 / 100, Loss: 2.1356
Epoch 80 / 100, Loss: 1.8393
Epoch 90 / 100, Loss: 1.6206


[I 2025-04-14 20:38:18,321] Trial 43 finished with value: 0.75177304964539 and parameters: {'K': 6, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.14527171269867037, 'model_type': 'GCNConv', 'batch_size': 1024, 'sampling': 'sage'}. Best is trial 35 with value: 0.8872469956077923.


 - Test Metrics: Accuracy=0.9285, F1=0.8208, Recall=0.7775, Precision=0.8692
 - Validation Metrics: Accuracy=0.6023, F1=0.7518, Recall=0.6023, Precision=1.0000
F1 = 0.75 < 0.82, skipping ...
Done. Results written to citeseer_experimentations\citeseer_scar_val_1404203713.csv.
Average F1 over valid seeds: 0.7518 ± 0.0000
Running experiment with seed=654:
 - K=5, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.2857007482338332, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 4.1774
Epoch 10 / 100, Loss: 3.8617
Epoch 20 / 100, Loss: 3.4083
Epoch 30 / 100, Loss: 3.1079
Epoch 40 / 100, Loss: 2.7425
Epoch 50 / 100, Loss: 2.3892
Epoch 60 / 100, Loss: 2.1091
Epoch 70 / 100, Loss: 1.8715
Epoch 80 / 100, Loss: 1.6159
Epoch 90 / 100, Loss: 1.4211
 - Test Metrics: Accuracy=0.9291, F1=0.8363, Recall=0.8602, Precision=0.8138
 - Val

[W 2025-04-14 20:41:59,154] Trial 44 failed with parameters: {'K': 5, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.2857007482338332, 'model_type': 'GCNConv', 'batch_size': 1024, 'sampling': 'sage'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_7376\455685410.py", line 38, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 560, in run_nnif_gnn_experiment
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 332, in train_graph
    scaler.scale(loss).backward()
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-package

KeyboardInterrupt: 

### Hyperparameter Optimization Citeseer
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 7),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "citeseer_sar_sampling_nnif_batch_cluster.csv",
        "min":0.845,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-25 10:10:43,640] A new study created in memory with name: no-name-f25fde7d-ee4a-4fa2-be4f-f9e486b8970e


Running experiment with seed=654:
 - K=3, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.11307195638376069, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pairs=1, clusters=50, lr=0.0023234250431828636
Epoch 0 / 100, Loss: 2.8385
Epoch 10 / 100, Loss: 2.4108
Epoch 20 / 100, Loss: 2.2628
Epoch 30 / 100, Loss: 2.1825
Epoch 40 / 100, Loss: 2.0409
Epoch 50 / 100, Loss: 1.9945
Epoch 60 / 100, Loss: 1.8232
Epoch 70 / 100, Loss: 1.6971
Epoch 80 / 100, Loss: 1.6468
Epoch 90 / 100, Loss: 1.4848
 - Metrics: Accuracy=0.9405, F1=0.8507, Recall=0.8046, Precision=0.9024
Running experiment with seed=114:
 - K=3, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.11307195638376069, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pa

[I 2025-03-25 10:13:49,885] Trial 0 finished with value: 0.857595508989174 and parameters: {'K': 3, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.11307195638376069, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0023234250431828636, 'anomaly_detector': 'nearest_neighbors', 'clusters': 50, 'num_epochs': 100, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 0 with value: 0.857595508989174.


 - Metrics: Accuracy=0.9390, F1=0.8493, Recall=0.8160, Precision=0.8854
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503101043.csv.
Average F1 over valid seeds: 0.8576 ± 0.0077
Running experiment with seed=654:
 - K=7, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=256, methodology=ours
 - ratio=0.17225386531043968, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=nearest_neighbors, sampling_k=25
 - model_type=GATConv, rate_pairs=2, clusters=400, lr=0.008752505664905935
Epoch 0 / 150, Loss: 42.4167
Epoch 10 / 150, Loss: 18.6557
Epoch 20 / 150, Loss: 10.0401
Epoch 30 / 150, Loss: 6.0609
Epoch 40 / 150, Loss: 4.2251
Epoch 50 / 150, Loss: 3.0423
Epoch 60 / 150, Loss: 2.2833
Epoch 70 / 150, Loss: 1.8259
Epoch 80 / 150, Loss: 1.4801
Epoch 90 / 150, Loss: 1.2157
Epoch 100 / 150, Loss: 1.0309
Epoch 110 / 150, Loss: 0.8732
Epoch 120 / 150, Loss: 0.7590
Epoch 130 / 150, Loss: 0.6525
Epoch 140 / 150, Loss

[I 2025-03-25 10:15:40,677] Trial 1 finished with value: 0.6813996316758748 and parameters: {'K': 7, 'layers': 2, 'hidden_channels': 256, 'out_channels': 64, 'ratio': 0.17225386531043968, 'model_type': 'GATConv', 'batch_size': 256, 'lr': 0.008752505664905935, 'anomaly_detector': 'unweighted', 'clusters': 400, 'num_epochs': 150, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 0 with value: 0.857595508989174.


...

 - Metrics: Accuracy=0.9059, F1=0.7183, Recall=0.5692, Precision=0.9732
F1 = 0.72 < 0.845, skipping ...
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503140907.csv.
Average F1 over valid seeds: 0.7183 ± 0.0000
Running experiment with seed=654:
 - K=2, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.15841141409782936, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.004733199465670689
Epoch 0 / 100, Loss: 2.1699
Epoch 10 / 100, Loss: 1.6532
Epoch 20 / 100, Loss: 1.4729
Epoch 30 / 100, Loss: 1.3237
Epoch 40 / 100, Loss: 1.1338
Epoch 50 / 100, Loss: 1.0336
Epoch 60 / 100, Loss: 0.8867
Epoch 70 / 100, Loss: 0.7703
Epoch 80 / 100, Loss: 0.7087
Epoch 90 / 100, Loss: 0.6093
 - Metrics: Accuracy=0.9408, F1=0.8600, Recall=0.8631, Precision=0.8569
Running experiment with seed=114:
 - K=2

[I 2025-03-25 14:12:40,226] Trial 98 finished with value: 0.8617602498928539 and parameters: {'K': 2, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.15841141409782936, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.004733199465670689, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 100, 'sampling': 'neighbor', 'sampling_k': 25}. Best is trial 87 with value: 0.8650138572197479.


 - Metrics: Accuracy=0.9387, F1=0.8539, Recall=0.8502, Precision=0.8576
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503140941.csv.
Average F1 over valid seeds: 0.8618 ± 0.0066
Running experiment with seed=654:
 - K=2, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.16102039029803536, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.004605584968207975
Epoch 0 / 100, Loss: 4.1609
Epoch 10 / 100, Loss: 3.4473
Epoch 20 / 100, Loss: 3.0725
Epoch 30 / 100, Loss: 2.6641
Epoch 40 / 100, Loss: 2.3129
Epoch 50 / 100, Loss: 2.1049
Epoch 60 / 100, Loss: 1.7786
Epoch 70 / 100, Loss: 1.6623
Epoch 80 / 100, Loss: 1.4632
Epoch 90 / 100, Loss: 1.3323


[I 2025-03-25 14:13:15,258] Trial 99 finished with value: 0.8429629629629629 and parameters: {'K': 2, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.16102039029803536, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.004605584968207975, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 100, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 87 with value: 0.8650138572197479.


 - Metrics: Accuracy=0.9363, F1=0.8430, Recall=0.8117, Precision=0.8767
F1 = 0.84 < 0.845, skipping ...
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503141240.csv.
Average F1 over valid seeds: 0.8430 ± 0.0000
Best trial:
  Average F1: 0.8650138572197479
  Best parameters:
    K: 2
    layers: 1
    hidden_channels: 128
    out_channels: 256
    ratio: 0.1611386183555754
    model_type: GCNConv
    batch_size: 1024
    lr: 0.0063212510924867535
    anomaly_detector: nearest_neighbors
    clusters: 200
    num_epochs: 100
    sampling: neighbor
    sampling_k: 25


### Hyperparameter Optimization Cora
#### SCAR 

In [9]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 1, 12),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.38),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [1024,2048]),
        "lr":0.005,# trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "cora_scar_val.csv",
        "min":0.85,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["sage"]),#,"cluster"])#,"shine"]),         
        
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-14 13:51:43,721] A new study created in memory with name: no-name-8ba802fb-56c4-4a9d-85d1-e6f495052f37


Running experiment with seed=654:
 - K=11, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.23223328947882702, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 3.0045
Epoch 10 / 100, Loss: 2.9991
Epoch 20 / 100, Loss: 2.8185
Epoch 30 / 100, Loss: 2.6127
Epoch 40 / 100, Loss: 2.7608
Epoch 50 / 100, Loss: 2.6592
Epoch 60 / 100, Loss: 2.4708
Epoch 70 / 100, Loss: 2.3688
Epoch 80 / 100, Loss: 2.2669
Epoch 90 / 100, Loss: 2.1285


[I 2025-04-14 13:52:02,212] Trial 0 finished with value: 0.7636363636363637 and parameters: {'K': 11, 'layers': 2, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.23223328947882702, 'model_type': 'GCNConv', 'batch_size': 2048, 'sampling': 'sage'}. Best is trial 0 with value: 0.7636363636363637.


 - Test Metrics: Accuracy=0.9165, F1=0.8519, Recall=0.7946, Precision=0.9181
 - Validation Metrics: Accuracy=0.6176, F1=0.7636, Recall=0.6176, Precision=1.0000
F1 = 0.76 < 0.85, skipping ...
Done. Results written to cora_experimentations\cora_scar_val_1404135143.csv.
Average F1 over valid seeds: 0.7636 ± 0.0000
Running experiment with seed=654:
 - K=3, layers=1, hidden=128, out=64
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.34056010611247234, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 2.8227
Epoch 10 / 100, Loss: 2.4395
Epoch 20 / 100, Loss: 2.1991
Epoch 30 / 100, Loss: 2.0069
Epoch 40 / 100, Loss: 1.8288
Epoch 50 / 100, Loss: 1.6486
Epoch 60 / 100, Loss: 1.4696
Epoch 70 / 100, Loss: 1.3386
Epoch 80 / 100, Loss: 1.2106
Epoch 90 / 100, Loss: 1.0893
 - Test Metrics: Accuracy=0.9114, F1=0.8531, Recall=0.8521, Precision=0.8542
 - Validation 

[I 2025-04-14 13:53:25,645] Trial 1 finished with value: 0.8776257444269465 and parameters: {'K': 3, 'layers': 1, 'hidden_channels': 128, 'out_channels': 64, 'ratio': 0.34056010611247234, 'model_type': 'GCNConv', 'batch_size': 1024, 'sampling': 'sage'}. Best is trial 1 with value: 0.8776257444269465.


...

 - Test Metrics: Accuracy=0.9243, F1=0.8778, Recall=0.8998, Precision=0.8568
 - Validation Metrics: Accuracy=0.7941, F1=0.8852, Recall=0.7941, Precision=1.0000
Done. Results written to cora_experimentations\cora_scar_val_1404182217.csv.
Average F1 over valid seeds: 0.9259 ± 0.0219
Running experiment with seed=654:
 - K=4, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.3794321121535841, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 3.1544
Epoch 10 / 100, Loss: 2.8051
Epoch 20 / 100, Loss: 2.6026
Epoch 30 / 100, Loss: 2.3915
Epoch 40 / 100, Loss: 2.1850
Epoch 50 / 100, Loss: 1.9736
Epoch 60 / 100, Loss: 1.7904
Epoch 70 / 100, Loss: 1.6152
Epoch 80 / 100, Loss: 1.4642
Epoch 90 / 100, Loss: 1.2912
 - Test Metrics: Accuracy=0.9258, F1=0.8811, Recall=0.9108, Precision=0.8534
 - Validation Metrics: Accuracy=0.8529, F1=0.9

[I 2025-04-14 18:29:11,135] Trial 163 finished with value: 0.9245043774050149 and parameters: {'K': 4, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.3794321121535841, 'model_type': 'GCNConv', 'batch_size': 1024, 'sampling': 'sage'}. Best is trial 95 with value: 0.9266950990544993.


 - Test Metrics: Accuracy=0.9254, F1=0.8796, Recall=0.9022, Precision=0.8581
 - Validation Metrics: Accuracy=0.7745, F1=0.8729, Recall=0.7745, Precision=1.0000
Done. Results written to cora_experimentations\cora_scar_val_1404182540.csv.
Average F1 over valid seeds: 0.9245 ± 0.0274
Running experiment with seed=654:
 - K=4, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.3789997924360248, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005


[W 2025-04-14 18:29:12,413] Trial 164 failed with parameters: {'K': 4, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.3789997924360248, 'model_type': 'GCNConv', 'batch_size': 1024, 'sampling': 'sage'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_7376\103825284.py", line 38, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 560, in run_nnif_gnn_experiment
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 297, in train_graph
    neg_mask, pos_mask = nnif_detector.get_reliable(features_np, y_labels)
                        

KeyboardInterrupt: 

### Hyperparameter Optimization Cora
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 7, 9),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.34),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [5,10,15]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "cora_sar_sampling_nnif_batch_cluster.csv",
        "min":0.85,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":"cluster",#trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-26 15:51:16,342] A new study created in memory with name: no-name-8176da62-abb1-46e4-8807-5e5ff1459ce0


Running experiment with seed=654:
 - K=9, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.16889523261113468, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.007646163484926425
Epoch 0 / 150, Loss: 3.4303
Epoch 10 / 150, Loss: 3.1501
Epoch 20 / 150, Loss: 2.8808
Epoch 30 / 150, Loss: 2.6330
Epoch 40 / 150, Loss: 2.3545
Epoch 50 / 150, Loss: 2.0481
Epoch 60 / 150, Loss: 1.7211
Epoch 70 / 150, Loss: 1.4545
Epoch 80 / 150, Loss: 1.2300
Epoch 90 / 150, Loss: 1.0373
Epoch 100 / 150, Loss: 0.8788
Epoch 110 / 150, Loss: 0.7530
Epoch 120 / 150, Loss: 0.6579
Epoch 130 / 150, Loss: 0.5790
Epoch 140 / 150, Loss: 0.5190
 - Metrics: Accuracy=0.9228, F1=0.8656, Recall=0.8227, Precision=0.9132
Running experiment with seed=114:
 - K=9, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.1688952

[I 2025-03-26 15:53:50,635] Trial 0 finished with value: 0.8684716037566916 and parameters: {'K': 9, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.16889523261113468, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.007646163484926425, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 150, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 0 with value: 0.8684716037566916.


 - Metrics: Accuracy=0.9269, F1=0.8731, Recall=0.8325, Precision=0.9178
Done. Results written to cora_experimentations\cora_sar_sampling_nnif_batch_cluster_2603155116.csv.
Average F1 over valid seeds: 0.8685 ± 0.0049
Running experiment with seed=654:
 - K=9, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.1886114414198443, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=nearest_neighbors, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=200, lr=0.00016096248213070527
Epoch 0 / 150, Loss: 35.3035
Epoch 10 / 150, Loss: 35.6685
Epoch 20 / 150, Loss: 34.8467
Epoch 30 / 150, Loss: 34.7338
Epoch 40 / 150, Loss: 34.8928
Epoch 50 / 150, Loss: 34.1160
Epoch 60 / 150, Loss: 33.9046
Epoch 70 / 150, Loss: 33.9726
Epoch 80 / 150, Loss: 33.1283
Epoch 90 / 150, Loss: 33.2420
Epoch 100 / 150, Loss: 32.6670
Epoch 110 / 150, Loss: 32.7155
[Early Stopping] at epoch 118


[I 2025-03-26 15:54:30,723] Trial 1 finished with value: 0.6825775656324582 and parameters: {'K': 9, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.1886114414198443, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.00016096248213070527, 'anomaly_detector': 'unweighted', 'clusters': 200, 'num_epochs': 150, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 0 with value: 0.8684716037566916.


...

Epoch 0 / 100, Loss: 5.9769
Epoch 10 / 100, Loss: 5.1580
Epoch 20 / 100, Loss: 4.5073
Epoch 30 / 100, Loss: 3.6713
Epoch 40 / 100, Loss: 3.1895
Epoch 50 / 100, Loss: 2.3100
Epoch 60 / 100, Loss: 1.8741
Epoch 70 / 100, Loss: 1.5376
Epoch 80 / 100, Loss: 1.2265
Epoch 90 / 100, Loss: 1.0520
 - Metrics: Accuracy=0.9210, F1=0.8694, Recall=0.8704, Precision=0.8683
Running experiment with seed=25:
 - K=8, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.2573033125681119, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=100, lr=0.007397697366787508


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 5.8094


[W 2025-03-26 16:49:22,637] Trial 31 failed with parameters: {'K': 8, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.2573033125681119, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.007397697366787508, 'anomaly_detector': 'nearest_neighbors', 'clusters': 100, 'num_epochs': 100, 'sampling': 'cluster', 'sampling_k': 25} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_39988\45902056.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 565, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
          

KeyboardInterrupt: 

### Hyperparameter Optimization Pubmed
#### SCAR 

In [8]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 20),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.2, 0.5),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [1024,2048]),
        "lr":0.005,# trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector": "nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "pubmed_scar_val.csv",
        "min":0.83,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["sage"]),#,"cluster"])#,"shine"]),         
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-13 22:10:30,338] A new study created in memory with name: no-name-ecfbfe14-7d22-4821-ab80-128040cd975e


Running experiment with seed=654:
 - K=20, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.20556970424026802, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 23.2672
Epoch 10 / 100, Loss: 20.2388
Epoch 20 / 100, Loss: 17.4405
Epoch 30 / 100, Loss: 13.5077
Epoch 40 / 100, Loss: 9.5019
Epoch 50 / 100, Loss: 6.4848
Epoch 60 / 100, Loss: 4.6085
Epoch 70 / 100, Loss: 3.4551
Epoch 80 / 100, Loss: 2.6916
Epoch 90 / 100, Loss: 2.1744


[I 2025-04-13 22:13:36,080] Trial 0 finished with value: 0.42685851318944845 and parameters: {'K': 20, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.20556970424026802, 'model_type': 'GATConv', 'batch_size': 2048, 'sampling': 'sage'}. Best is trial 0 with value: 0.42685851318944845.


 - Test Metrics: Accuracy=0.8084, F1=0.6946, Recall=0.5455, Precision=0.9559
 - Validation Metrics: Accuracy=0.2713, F1=0.4269, Recall=0.2713, Precision=1.0000
F1 = 0.43 < 0.83, skipping ...
Done. Results written to pubmed_experimentations\pubmed_scar_val_1304221030.csv.
Average F1 over valid seeds: 0.4269 ± 0.0000
Running experiment with seed=654:
 - K=3, layers=1, hidden=64, out=128
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.23957585696143865, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 9.8411
Epoch 10 / 100, Loss: 6.9378
Epoch 20 / 100, Loss: 4.9175
Epoch 30 / 100, Loss: 3.5438
Epoch 40 / 100, Loss: 2.6459
Epoch 50 / 100, Loss: 2.0335
Epoch 60 / 100, Loss: 1.6190
Epoch 70 / 100, Loss: 1.3083
Epoch 80 / 100, Loss: 1.0882
Epoch 90 / 100, Loss: 0.9177


[I 2025-04-13 22:15:31,618] Trial 1 finished with value: 0.533532041728763 and parameters: {'K': 3, 'layers': 1, 'hidden_channels': 64, 'out_channels': 128, 'ratio': 0.23957585696143865, 'model_type': 'GCNConv', 'batch_size': 2048, 'sampling': 'sage'}. Best is trial 1 with value: 0.533532041728763.


...

 - Test Metrics: Accuracy=0.8748, F1=0.8505, Recall=0.8913, Precision=0.8132
 - Validation Metrics: Accuracy=0.8232, F1=0.9030, Recall=0.8232, Precision=1.0000
Done. Results written to pubmed_experimentations\pubmed_scar_val_1404131815.csv.
Average F1 over valid seeds: 0.8979 ± 0.0063
Running experiment with seed=654:
 - K=12, layers=2, hidden=64, out=256
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.4905369390516624, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 19.7314
Epoch 10 / 100, Loss: 16.8071
Epoch 20 / 100, Loss: 12.9634
Epoch 30 / 100, Loss: 9.0839
Epoch 40 / 100, Loss: 6.3053
Epoch 50 / 100, Loss: 4.5078
Epoch 60 / 100, Loss: 3.3906
Epoch 70 / 100, Loss: 2.6435
Epoch 80 / 100, Loss: 2.1255
Epoch 90 / 100, Loss: 1.7483
 - Test Metrics: Accuracy=0.8455, F1=0.8155, Recall=0.8549, Precision=0.7795
 - Validation Metrics: Accuracy=0.7642

[I 2025-04-14 13:40:17,997] Trial 69 finished with value: 0.8396935956374066 and parameters: {'K': 12, 'layers': 2, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.4905369390516624, 'model_type': 'GCNConv', 'batch_size': 2048, 'sampling': 'sage'}. Best is trial 68 with value: 0.8979370412607548.


 - Test Metrics: Accuracy=0.8148, F1=0.7778, Recall=0.8113, Precision=0.7469
 - Validation Metrics: Accuracy=0.6850, F1=0.8130, Recall=0.6850, Precision=1.0000
F1 = 0.81 < 0.83, skipping ...
Done. Results written to pubmed_experimentations\pubmed_scar_val_1404133446.csv.
Average F1 over valid seeds: 0.8397 ± 0.0267
Running experiment with seed=654:
 - K=14, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.48692665775241023, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 20.9056
Epoch 10 / 100, Loss: 17.9111
Epoch 20 / 100, Loss: 14.2243
Epoch 30 / 100, Loss: 10.0597
Epoch 40 / 100, Loss: 6.9475
Epoch 50 / 100, Loss: 4.9040
Epoch 60 / 100, Loss: 3.6425
Epoch 70 / 100, Loss: 2.8155
Epoch 80 / 100, Loss: 2.2480
Epoch 90 / 100, Loss: 1.8375
 - Test Metrics: Accuracy=0.8679, F1=0.8387, Recall=0.8599, Precision=0.8186
 - V

[W 2025-04-14 13:51:35,254] Trial 70 failed with parameters: {'K': 14, 'layers': 2, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.48692665775241023, 'model_type': 'GCNConv', 'batch_size': 2048, 'sampling': 'sage'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_7376\296208695.py", line 37, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 560, in run_nnif_gnn_experiment
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 326, in train_graph
    lp_loss, E = lp_criterion(sub_emb, sub_adj, sub_pos, sub_neg)
                 ^^^^^^^^^^^^^^

KeyboardInterrupt: 

### Hyperparameter Optimization Pubmed
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 12),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.2, 0.4),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [15,256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "pubmed_sar_sampling_nnif_batch_cluster.csv",
        "min":0.84,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-24 05:51:18,349] A new study created in memory with name: no-name-7363eccd-5b06-40dc-87b0-25014df8c5a1


Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.3362908893186948, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=cluster, sampling_k=50
 - model_type=GATConv, rate_pairs=10, clusters=300, lr=0.0007087214622711031


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 18.5522
Epoch 10 / 100, Loss: 17.8089
Epoch 20 / 100, Loss: 17.3418
Epoch 30 / 100, Loss: 16.8934
Epoch 40 / 100, Loss: 16.1184
Epoch 50 / 100, Loss: 15.1700
Epoch 60 / 100, Loss: 14.4180
Epoch 70 / 100, Loss: 13.4842
Epoch 80 / 100, Loss: 12.6036
Epoch 90 / 100, Loss: 11.9384


[I 2025-03-24 05:53:00,820] Trial 0 finished with value: 0.715743932941256 and parameters: {'K': 7, 'ratio': 0.3362908893186948, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0007087214622711031, 'anomaly_detector': 'unweighted', 'clusters': 300, 'num_epochs': 100, 'sampling': 'cluster', 'sampling_k': 50}. Best is trial 0 with value: 0.715743932941256.


...

 - Metrics: Accuracy=0.8745, F1=0.8391, Recall=0.8193, Precision=0.8598
F1 = 0.84 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_sar_sampling_nnif_batch_cluster_2403133828.csv.
Average F1 over valid seeds: 0.8391 ± 0.0000
Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.28001597838526016, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=50
 - model_type=GCNConv, rate_pairs=10, clusters=400, lr=0.00873922908266354
Epoch 0 / 50, Loss: 58.0806
Epoch 10 / 50, Loss: 8.0599
Epoch 20 / 50, Loss: 2.8407
Epoch 30 / 50, Loss: 1.4709
Epoch 40 / 50, Loss: 0.8989
 - Metrics: Accuracy=0.8762, F1=0.8413, Recall=0.8212, Precision=0.8624
Running experiment with seed=114:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.28001597838526016, aggregation=mean, treatmen

[I 2025-03-24 15:01:39,121] Trial 13 finished with value: 0.835498509343777 and parameters: {'K': 7, 'ratio': 0.28001597838526016, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.00873922908266354, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 50, 'sampling': 'nearest_neighbors', 'sampling_k': 50}. Best is trial 10 with value: 0.8395798210348853.


 - Metrics: Accuracy=0.8702, F1=0.8297, Recall=0.7916, Precision=0.8716
F1 = 0.83 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_sar_sampling_nnif_batch_cluster_2403141459.csv.
Average F1 over valid seeds: 0.8355 ± 0.0058
Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.26308150498044675, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=50
 - model_type=GCNConv, rate_pairs=10, clusters=400, lr=0.009817622280543212
Epoch 0 / 50, Loss: 57.2382
Epoch 10 / 50, Loss: 6.7150
Epoch 20 / 50, Loss: 2.3497


[W 2025-03-24 15:11:12,619] Trial 14 failed with parameters: {'K': 7, 'ratio': 0.26308150498044675, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.009817622280543212, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 50, 'sampling': 'nearest_neighbors', 'sampling_k': 50} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_22248\4249684348.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 565, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
                                              ^^^^^^^^^^^

KeyboardInterrupt: 

### Hyperparameter Optimization WikiCS
#### SCAR 

In [7]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 35),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.15, 0.35),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs",1,15),
        "batch_size": trial.suggest_categorical("batch_size", [1024,2048]),
        "lr": 0.005,
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "wikics_scar_val.csv",
        "min":0.88,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,      
        "sampling":"sage",#trial.suggest_categorical("sampling",["sage"])#,"cluster"])#,"shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-13 16:09:40,134] A new study created in memory with name: no-name-0d2c8ed0-3aee-4335-813f-92fc53088de8


Running experiment with seed=654:
 - K=22, layers=1, hidden=64, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.16728924746965734, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 14.4396
Epoch 10 / 100, Loss: 13.0222
Epoch 20 / 100, Loss: 11.7840
Epoch 30 / 100, Loss: 10.9750
Epoch 40 / 100, Loss: 9.6940
Epoch 50 / 100, Loss: 8.6542
Epoch 60 / 100, Loss: 7.3962
Epoch 70 / 100, Loss: 6.0884
Epoch 80 / 100, Loss: 4.9631
Epoch 90 / 100, Loss: 4.0282


[I 2025-04-13 16:10:45,488] Trial 0 finished with value: 0.6437246963562753 and parameters: {'K': 22, 'layers': 1, 'hidden_channels': 64, 'out_channels': 64, 'ratio': 0.16728924746965734, 'model_type': 'GCNConv', 'batch_size': 2048}. Best is trial 0 with value: 0.6437246963562753.


 - Test Metrics: Accuracy=0.9080, F1=0.7767, Recall=0.6991, Precision=0.8736
 - Validation Metrics: Accuracy=0.4746, F1=0.6437, Recall=0.4746, Precision=1.0000
F1 = 0.64 < 0.88, skipping ...
Done. Results written to wiki-cs_experimentations\wikics_scar_val_1304160940.csv.
Average F1 over valid seeds: 0.6437 ± 0.0000
Running experiment with seed=654:
 - K=11, layers=2, hidden=128, out=128
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.2267092003974478, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 20.5200
Epoch 10 / 100, Loss: 16.7326
Epoch 20 / 100, Loss: 11.5211
Epoch 30 / 100, Loss: 7.2435
Epoch 40 / 100, Loss: 4.8031
Epoch 50 / 100, Loss: 3.4190
Epoch 60 / 100, Loss: 2.5533
Epoch 70 / 100, Loss: 1.9905
Epoch 80 / 100, Loss: 1.6060
Epoch 90 / 100, Loss: 1.3150


[I 2025-04-13 16:13:00,840] Trial 1 finished with value: 0.8368055555555556 and parameters: {'K': 11, 'layers': 2, 'hidden_channels': 128, 'out_channels': 128, 'ratio': 0.2267092003974478, 'model_type': 'GATConv', 'batch_size': 1024}. Best is trial 1 with value: 0.8368055555555556.


...

 - Test Metrics: Accuracy=0.9138, F1=0.8375, Recall=0.9709, Precision=0.7364
 - Validation Metrics: Accuracy=0.9463, F1=0.9724, Recall=0.9463, Precision=1.0000
Done. Results written to wiki-cs_experimentations\wikics_scar_val_1304215247.csv.
Average F1 over valid seeds: 0.9717 ± 0.0054
Running experiment with seed=654:
 - K=27, layers=2, hidden=128, out=128
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.3448333149058221, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 15.1803
Epoch 10 / 100, Loss: 13.7606
Epoch 20 / 100, Loss: 12.6243
Epoch 30 / 100, Loss: 11.5799
Epoch 40 / 100, Loss: 10.5519
Epoch 50 / 100, Loss: 9.6783
Epoch 60 / 100, Loss: 8.5280
Epoch 70 / 100, Loss: 7.3411
Epoch 80 / 100, Loss: 6.0512
Epoch 90 / 100, Loss: 4.8803
 - Test Metrics: Accuracy=0.9524, F1=0.8980, Recall=0.9156, Precision=0.8811
 - Validation Metrics: Accuracy=0.

[I 2025-04-13 22:04:52,178] Trial 69 finished with value: 0.9286110191469696 and parameters: {'K': 27, 'layers': 2, 'hidden_channels': 128, 'out_channels': 128, 'ratio': 0.3448333149058221, 'model_type': 'GCNConv', 'batch_size': 2048}. Best is trial 54 with value: 0.9764390721867331.


 - Test Metrics: Accuracy=0.9549, F1=0.9044, Recall=0.9321, Precision=0.8783
 - Validation Metrics: Accuracy=0.8746, F1=0.9331, Recall=0.8746, Precision=1.0000
Done. Results written to wiki-cs_experimentations\wikics_scar_val_1304215835.csv.
Average F1 over valid seeds: 0.9286 ± 0.0090
Running experiment with seed=654:
 - K=8, layers=1, hidden=128, out=128
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.32904692516640466, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 18.7026
Epoch 10 / 100, Loss: 14.3659
Epoch 20 / 100, Loss: 9.5629
Epoch 30 / 100, Loss: 6.2184
Epoch 40 / 100, Loss: 4.2414
Epoch 50 / 100, Loss: 3.0367
Epoch 60 / 100, Loss: 2.3163
Epoch 70 / 100, Loss: 1.8156
Epoch 80 / 100, Loss: 1.4719
Epoch 90 / 100, Loss: 1.2221
 - Test Metrics: Accuracy=0.9228, F1=0.8461, Recall=0.9265, Precision=0.7785
 - Validation Metrics: Accuracy=0.886

[W 2025-04-13 22:10:15,812] Trial 70 failed with parameters: {'K': 8, 'layers': 1, 'hidden_channels': 128, 'out_channels': 128, 'ratio': 0.32904692516640466, 'model_type': 'GATConv', 'batch_size': 1024} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_7376\368764763.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 560, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
                                              ^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 389, in train_graph


KeyboardInterrupt: 

### Hyperparameter Optimization WikiCS
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 3, 40),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,10),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "wikics_sar_sampling_nnif_batch_cluster.csv",
        "min":0.88,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


# Elliptic Bitcoin

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "elliptic-bitcoin",      
        "mechanism": "SAR2",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": trial.suggest_int("K", 2, 20),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": trial.suggest_float("dropout", 0, 0.4),
        "ratio": trial.suggest_float("ratio", 0.001, 0.03),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,15),
        "batch_size": trial.suggest_categorical("batch_size", [256, 512,768, 1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 1,
        "output_csv": "elliptic_bitcoin_cluster.csv",
        "min":0.65,
        "clusters":trial.suggest_categorical("clusters", [100,500,1000]),          
        "num_epochs":100,#trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":trial.suggest_categorical("sampling",["neighbor","cluster"]),         
        }

    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-04 10:31:12,718] A new study created in memory with name: no-name-1f8e694f-776b-45ff-92be-1b1c7abb1f96


Running experiment with seed=654:
 - K=16, layers=2, hidden=128, out=64
 - norm=None, dropout=0.042811745845246656, batch_size=20, methodology=ours
 - ratio=0.014408704706040928, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster
 - model_type=GCNConv, rate_pairs=13, clusters=500, lr=0.0021337433006543764


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 38.8043
Epoch 10 / 100, Loss: 35.0547
Epoch 20 / 100, Loss: 28.8067
Epoch 30 / 100, Loss: 21.0404
Epoch 40 / 100, Loss: 14.7131
Epoch 50 / 100, Loss: 10.2298
Epoch 60 / 100, Loss: 7.5048
Epoch 70 / 100, Loss: 5.7055
Epoch 80 / 100, Loss: 4.2609
Epoch 90 / 100, Loss: 3.3113


[I 2025-04-04 10:42:04,920] Trial 0 finished with value: 0.6484468737501666 and parameters: {'K': 16, 'layers': 2, 'hidden_channels': 128, 'out_channels': 64, 'dropout': 0.042811745845246656, 'ratio': 0.014408704706040928, 'model_type': 'GCNConv', 'rate_pairs': 13, 'batch_size': 20, 'lr': 0.0021337433006543764, 'clusters': 500}. Best is trial 0 with value: 0.6484468737501666.


...

Error: Labels must be in {0,1}, got [0].
Done. Results written to elliptic-bitcoin_experimentations\elliptic_bitcoin_cluster_0504152508.csv.
Average F1 over valid seeds: 0.0000 ± 0.0000
Running experiment with seed=654:
 - K=4, layers=1, hidden=128, out=128
 - norm=None, dropout=0.027456769022691726, batch_size=20, methodology=ours
 - ratio=0.016695805654684717, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster
 - model_type=GCNConv, rate_pairs=6, clusters=500, lr=0.0024836631205545495


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 30.4772
Epoch 10 / 100, Loss: 19.1925
Epoch 20 / 100, Loss: 11.6362
Epoch 30 / 100, Loss: 7.9682
Epoch 40 / 100, Loss: 5.4701
Epoch 50 / 100, Loss: 4.0614
Epoch 60 / 100, Loss: 3.0453
Epoch 70 / 100, Loss: 2.3516
Epoch 80 / 100, Loss: 1.9151
Epoch 90 / 100, Loss: 1.4996


[I 2025-04-05 15:35:05,365] Trial 199 finished with value: 0.712346817762551 and parameters: {'K': 4, 'layers': 1, 'hidden_channels': 128, 'out_channels': 128, 'dropout': 0.027456769022691726, 'ratio': 0.016695805654684717, 'model_type': 'GCNConv', 'rate_pairs': 6, 'batch_size': 20, 'lr': 0.0024836631205545495, 'clusters': 500}. Best is trial 165 with value: 0.7249539836970813.


 - Metrics: Accuracy=0.9531, F1=0.7123, Recall=0.5947, Precision=0.8880
Done. Results written to elliptic-bitcoin_experimentations\elliptic_bitcoin_cluster_0504152543.csv.
Average F1 over valid seeds: 0.7123 ± 0.0000
Best trial:
  Average F1: 0.7249539836970813
  Best parameters:
    K: 4
    layers: 1
    hidden_channels: 128
    out_channels: 128
    dropout: 0.04553314815786919
    ratio: 0.01896077000764923
    model_type: GCNConv
    rate_pairs: 6
    batch_size: 20
    lr: 0.0033245734871396367
    clusters: 500
